# Feature Engineering

We remove duplicates rows including reversal transactionType and multi-swipe transactions.

In [31]:
df2=df.copy()

df2=df2.loc[df2.transactionType!='REVERSAL']
df2['MultiSwipe'] = (df2.sort_values(['transactionDateTime'])
                       .groupby(['customerId','merchantName', 'transactionAmount','transactionType'], sort=False)['transactionDateTime']
                       .diff()
                       .dt.total_seconds()
                       .lt(180))
df2=df2.loc[dft.MultiSwipe==False]
df2=df2.reset_index(drop=True)

df3=df2.copy()

#pd.reset_option(“max_columns”)
pd.set_option('display.max_columns', None)



Then we caculate percentage of Null values in each column:


In [32]:
round(df3.isnull().sum()*100/len(df3),2)

accountNumber               0.00
customerId                  0.00
creditLimit                 0.00
availableMoney              0.00
transactionDateTime         0.00
transactionAmount           0.00
merchantName                0.00
acqCountry                  0.58
merchantCountryCode         0.09
posEntryMode                0.51
posConditionCode            0.05
merchantCategoryCode        0.00
currentExpDate              0.00
accountOpenDate             0.00
dateOfLastAddressChange     0.00
cardCVV                     0.00
enteredCVV                  0.00
cardLast4Digits             0.00
transactionType             0.09
currentBalance              0.00
cardPresent                 0.00
expirationDateKeyInMatch    0.00
isFraud                     0.00
MultiSwipe                  0.00
dtype: float64


Since less than 0.6% of rows have null values, we removed those rows from dataset.


In [33]:
df3.dropna(inplace=True)

#df3.info()

In [34]:
#df3.head()

## Remove Irrelevant Columns

We remove these columns ('accountNumber','customerId','MultiSwipe','cardLast4Digits','currentBalance') because they do not affect our final prediction. 
Note that currentBalance=creditLimit-availableMoney and we already have "credit limit" and "available money" in our dataset, so we do not need "current balance" anymore.

In [35]:
df3.drop(['accountNumber','customerId','MultiSwipe','cardLast4Digits','currentBalance','merchantName'],axis=1,inplace=True)

## Create new features

We define a new feature as CVV_correct to see if a customer enters right CVV code. We can use this one later for our fraud detection model.Then we remove 'cardCVV' and 'enteredCVV' since we do not need them anymore.

In [36]:
df4=df3.copy()

df4['CVV_correct'] = df4['cardCVV'] == df4['enteredCVV']
df4.drop(['cardCVV','enteredCVV'],axis=1,inplace=True)

We now want to converted these 3 date columns to number as follows:
    - accountOpenDate_days: Number of days since its opening
    - dateOfLastAddressChange_days: Number of days since last address changing
    - currentExpDate_days: Number of days until expiration date


In [37]:
df4['accountOpenDate'] = pd.to_datetime(df4['accountOpenDate'])
df4['dateOfLastAddressChange'] = pd.to_datetime(df4['dateOfLastAddressChange'])
df4['currentExpDate'] = pd.to_datetime(df4['currentExpDate'])

df4['accountOpenDate_days']=pd.Timestamp.now().normalize()-df4['accountOpenDate']
df4['dateOfLastAddressChange_days']=pd.Timestamp.now().normalize()-df4['dateOfLastAddressChange']
df4['currentExpDate_days']=df4['currentExpDate']-pd.Timestamp.now().normalize()

df4['accountOpenDate_days'] = df4['accountOpenDate_days'].astype(int)
df4['accountOpenDate_days']=df4['accountOpenDate_days']/(24*3600*(10**9))
df4['dateOfLastAddressChange_days'] = df4['dateOfLastAddressChange_days'].astype(int)
df4['dateOfLastAddressChange_days']=df4['dateOfLastAddressChange_days']/(24*3600*(10**9))
df4['currentExpDate_days'] = df4['currentExpDate_days'].astype(int)
df4['currentExpDate_days']=df4['currentExpDate_days']/(24*3600*(10**9))

df4.drop(['accountOpenDate','dateOfLastAddressChange','currentExpDate'],axis=1,inplace=True)


Regarding 'transactionDateTime' column, we extracted 'day' and 'hour' to address the effect of transaction time on fraud.


In [38]:
df4['tran_day'] = df4['transactionDateTime'].dt.day
df4['tran_hour'] = df4['transactionDateTime'].dt.hour
df4.drop(['transactionDateTime'],axis=1,inplace=True)

#df4.head()

## Convert bool to int

We converted bool columns ('cardPresent','expirationDateKeyInMatch','isFraud','CVV_correct') to 0 and 1 (False=0 and True=1)

In [39]:
df5=df4.copy()

df5['cardPresent'] = df5['cardPresent'].astype(int)
df5['expirationDateKeyInMatch'] = df5['expirationDateKeyInMatch'].astype(int)
df5['isFraud'] = df5['isFraud'].astype(int)
df5['CVV_correct'] = df5['CVV_correct'].astype(int)

#df5.head()

## Categorical Columns

In [40]:
df6=df5.copy()

### 1. acqCountry Column

In [41]:
df6['acqCountry'].value_counts()

US     742228
MEX      2997
CAN      2311
PR       1482
Name: acqCountry, dtype: int64

So we use One-hot Encoding for this feature

In [42]:
df6 = pd.concat([df6,pd.get_dummies(df6['acqCountry'], prefix='acqCountry')], axis=1).drop(['acqCountry'],axis=1)


### 2. merchantCountryCode Column

In [43]:
df6['merchantCountryCode'].value_counts()

US     742230
MEX      2993
CAN      2305
PR       1490
Name: merchantCountryCode, dtype: int64

So we use One-hot Encoding for this feature

In [44]:
df6 = pd.concat([df6,pd.get_dummies(df6['merchantCountryCode'], prefix='merchantCountryCode')], axis=1).drop(['merchantCountryCode'],axis=1)


### 3. posEntryMode Column

In [45]:
df6['posEntryMode'].value_counts()

05    301719
09    226279
02    187588
90     18775
80     14657
Name: posEntryMode, dtype: int64

So we use One-hot Encoding for this feature

In [46]:
df6 = pd.concat([df6,pd.get_dummies(df6['posEntryMode'], prefix='posEntryMode')], axis=1).drop(['posEntryMode'],axis=1)


### 4. posConditionCode Column

In [47]:
df6['posConditionCode'].value_counts()

01    599165
08    142683
99      7170
Name: posConditionCode, dtype: int64

So we use One-hot Encoding for this feature

In [48]:
df6 = pd.concat([df6,pd.get_dummies(df6['posConditionCode'], prefix='posConditionCode')], axis=1).drop(['posConditionCode'],axis=1)


### 5. merchantCategoryCode Column

In [49]:
df6['merchantCategoryCode'].value_counts()

online_retail           192037
fastfood                106443
entertainment            76050
food                     71595
online_gifts             62914
rideshare                48630
hotels                   32331
fuel                     23631
subscriptions            21789
auto                     20555
health                   18136
personal care            17998
mobileapps               14787
airline                  14581
online_subscriptions     10928
furniture                 7080
food_delivery             6000
gym                       2183
cable/phone               1350
Name: merchantCategoryCode, dtype: int64

So we use One-hot Encoding for this feature

In [50]:
df6 = pd.concat([df6,pd.get_dummies(df6['merchantCategoryCode'], prefix='merchantCategoryCode')], axis=1).drop(['merchantCategoryCode'],axis=1)


Before ending feature eng section, we can see for "ADDRESS_VERIFICATION" transaction, there are just 112 fraud so in order to prevent biasing the model, we just work with 'PURCHASE' transactions.

In [51]:
df7=df6.loc[df6.transactionType=='ADDRESS_VERIFICATION']
df7['isFraud'].value_counts()

0    19637
1      112
Name: isFraud, dtype: int64

In [52]:
df6=df6.loc[df6.transactionType=='PURCHASE']
df6.drop(['transactionType'],axis=1,inplace=True)

#df6.head()